In [1]:
import torch
import os
from monai.transforms import (  LoadImaged,
                                EnsureChannelFirstd,
                                Compose,
                                ScaleIntensityRanged,
                                CropForegroundd,
                                Orientationd,
                                Spacingd,
                                Spacing,
                                NormalizeIntensity,)

from monai.transforms.compose import MapTransform
from nnunetv2.inference.predict_from_raw_data import load_what_we_need
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.inferers import sliding_window_inference

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [ ]:
Predicting...
(CUDA) Elapsed time:  402.07ms
(CUDA) Elapsed time:   66.62ms
(CUDA) Elapsed time:   66.05ms
(CUDA) Elapsed time:   65.52ms
(CUDA) Elapsed time:   63.91ms
(CUDA) Elapsed time:   63.46ms
(CUDA) Elapsed time:   63.56ms
(CUDA) Elapsed time:   64.41ms
  Predicted in 8.84s
Resampling...
Saving segmentations...
100%|██████████| 117/117 [00:04<00:00, 23.76it/s]
  Saved in 5.50s
Execution took 64.79±1.18ms over 7 function calls.
1 function call was ignored in this report due to warm up.
Dice Score: 0.949

Process finished with exit code 0


In [2]:
TS_ROOT = os.path.join(os.environ['HOME'], '.totalsegmentator/nnunet/results')
patch_size = (1, 112, 112, 128)
model_path = (
    'Dataset297_TotalSegmentator_total_3mm_1559subj/'
    'nnUNetTrainer_4000epochs_NoMirroring__nnUNetPlans__3d_fullres'
)
path = os.path.join(TS_ROOT, model_path)
data_d = [{"image": "/mnt/ssd/PycharmProjects/tucker-cnn/data/spleen/imagesTr/spleen_2.nii.gz"}]

In [3]:
mostly_useless_stuff = load_what_we_need(
        model_training_output_dir=path,
        use_folds=[0],
        checkpoint_name='checkpoint_final.pth',
    )
network = mostly_useless_stuff[-2]

In [18]:
class PreprocessNorm(MapTransform):
    """
    This transform class takes NNUNet's preprocessing method for reference.
    That code is in:
    https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunet/preprocessing/preprocessing.py

    """

    def __init__(
        self,
        keys,
        clip_values,
        normalize_values,
    ) -> None:
        super().__init__(keys)
        self.keys = keys
        self.low = clip_values[0]
        self.high = clip_values[1]
        self.mean = normalize_values[0]
        self.std = normalize_values[1]
        self.training = False
        self.normalize_intensity = NormalizeIntensity(nonzero=True, channel_wise=True)

    def __call__(self, data):
        # load data
        d = dict(data)
        image = d["image"]

        # clip image for CT dataset
        if self.low != 0 or self.high != 0:
            image = torch.clamp(image, self.low, self.high)
            image = (image - self.mean) / self.std
        else:
            image = self.normalize_intensity(image.copy())

        d["image"] = image
        return d

In [19]:
val_keys = ["image", "label"]
val_mode = ("bilinear", "nearest")
test_keys = ["image"]

test_transforms = Compose(
    [
        LoadImaged(keys=test_keys),
        EnsureChannelFirstd(keys=test_keys),
        PreprocessNorm(
            keys=test_keys,
            clip_values=(-1004.0, 1588.0),
            normalize_values=(-50.38697721419439, 503.39235619144),
        ),
        #CropForegroundd(keys=["image", "label"], source_key="image"),
        #Orientationd(keys=test_keys, axcodes="LAS"),
        Spacingd(keys=test_keys, pixdim=(3., 3., 3.), mode=("bilinear")), #pixdim=(1.5, 1.5, 1.5)
    ]
)

In [83]:
check_ds = Dataset(data=data_d, transform=test_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

In [84]:
im = check_ds.__getitem__(0)["image"]

In [85]:
im.shape

torch.Size([1, 512, 512, 90])

In [88]:
plt.imshow(im[0,:,:,60])

In [4]:
import numpy as np
import matplotlib.pylab as plt

In [15]:
a = np.load("seg_pred.npy")
b = np.load("seg_true.npy")

In [16]:
aa = torch.softmax(torch.from_numpy(a), dim=0)

RuntimeError: "softmax_kernel_impl" not implemented for 'Long'

In [13]:
aa.shape

NameError: name 'aa' is not defined

In [14]:
aa.max()

NameError: name 'aa' is not defined

In [ ]:
plt.imshow(a[0,:,:,60]>0.5)

In [5]:
from nnunetv2.preprocessing.preprocessors.default_preprocessor import DefaultPreprocessor
plans_manager = mostly_useless_stuff[3]
dataset_json_file = mostly_useless_stuff[4]
pp = DefaultPreprocessor()
data, _, properties = pp.run_case(["/mnt/ssd/PycharmProjects/tucker-cnn/data/spleen/imagesTr/spleen_2.nii.gz"], seg_file=None, plans_manager=plans_manager,
                                      configuration_manager=plans_manager.get_configuration('3d_fullres'),
                                      dataset_json=dataset_json_file)

old shape: (90, 512, 512), new_shape: [150 136 136], old_spacing: [5.0, 0.7949219942092896, 0.7949219942092896], new_spacing: [3.0, 3.0, 3.0], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7f281e6db4c0>, is_seg=False, order=3, order_z=0, force_separate_z=None)


In [7]:
plt.imshow(data[0, 70, :,:])
plt.show()

/tmp/ipykernel_755358/1157872989.py:2: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
def example_test_case_preprocessing():
    # (paths to files may need adaptations)
    plans_file = '/home/isensee/drives/gpu_data/nnUNet_preprocessed/Dataset219_AMOS2022_postChallenge_task2/nnUNetPlans.json'
    dataset_json_file = '/home/isensee/drives/gpu_data/nnUNet_preprocessed/Dataset219_AMOS2022_postChallenge_task2/dataset.json'
    input_images = ['/home/isensee/drives/e132-rohdaten/nnUNetv2/Dataset219_AMOS2022_postChallenge_task2/imagesTr/amos_0600_0000.nii.gz', ]  # if you only have one channel, you still need a list: ['case000_0000.nii.gz']

    configuration = '3d_fullres'
    pp = DefaultPreprocessor()

    # _ because this position would be the segmentation if seg_file was not None (training case)
    # even if you have the segmentation, don't put the file there! You should always evaluate in the original
    # resolution. What comes out of the preprocessor might have been resampled to some other image resolution (as
    # specified by plans)
    plans_manager = PlansManager(plans_file)
    data, _, properties = pp.run_case(input_images, seg_file=None, plans_manager=plans_manager,
                                      configuration_manager=plans_manager.get_configuration(configuration),
                                      dataset_json=dataset_json_file)

    # voila. Now plug data into your prediction function of choice. We of course recommend nnU-Net's default (TODO)
    return data